## 1. Import

In [1]:
from core.dataframe import Dataframe
from datetime  import timedelta

In [2]:
df=Dataframe.read_tdsx(r"./datasource/Base Propensão Antecipação - v2.tdsx") # passar o id da tabela no tableau para baixar

In [3]:
df.head(5)

,ANTECIPA,dias_desde_ultima_antecipacao,CNPJ,DATA,dias_ate_primeira_antecipacao,TPV_CRED,QTD_TRN UNICA,dias_desde_primeiro_tpv,SISTEMA,SALDO UNICA,UF,CIDADE,GERENTE,Região
0,0,1,24940522000131,2025-10-16,267.0,8496.80,113,912,NaN,316400.63,GO,MAMBAI,ANIZIO FILHO,CENTRO-OESTE
1,0,2,24940522000131,2025-10-15,266.0,12961.51,163,911,NaN,308631.62,GO,MAMBAI,ANIZIO FILHO,CENTRO-OESTE
2,0,4,24940522000131,2025-10-13,264.0,6551.58,107,909,NaN,244213.49,GO,MAMBAI,ANIZIO FILHO,CENTRO-OESTE
3,0,8,24940522000131,2025-10-09,260.0,9556.97,122,905,NaN,233943.55,GO,MAMBAI,ANIZIO FILHO,CENTRO-OESTE
4,0,11,24940522000131,2025-10-06,257.0,10247.04,92,902,NaN,200122.26,GO,MAMBAI,ANIZIO FILHO,CENTRO-OESTE


In [4]:

# Verificar informações do dataset
print("Informações do dataset:")
print("Shape do dataset:", df.shape)

print("\nDistribuição da variável target:")
print(df['ANTECIPA'].value_counts())
print("=" * 34)

Informações do dataset:
Shape do dataset: (435042, 14)

Distribuição da variável target:
ANTECIPA
0    410148
1     24894
Name: count, dtype: int64


In [5]:
print(df["DATA"].max())

2025-10-17


In [6]:
df = df.drop("SISTEMA", axis=1)
df.fillna(0,inplace=True)
df.head(5)

,ANTECIPA,dias_desde_ultima_antecipacao,CNPJ,DATA,dias_ate_primeira_antecipacao,TPV_CRED,QTD_TRN UNICA,dias_desde_primeiro_tpv,SALDO UNICA,UF,CIDADE,GERENTE,Região
0,0,1,24940522000131,2025-10-16,267.0,8496.80,113,912,316400.63,GO,MAMBAI,ANIZIO FILHO,CENTRO-OESTE
1,0,2,24940522000131,2025-10-15,266.0,12961.51,163,911,308631.62,GO,MAMBAI,ANIZIO FILHO,CENTRO-OESTE
2,0,4,24940522000131,2025-10-13,264.0,6551.58,107,909,244213.49,GO,MAMBAI,ANIZIO FILHO,CENTRO-OESTE
3,0,8,24940522000131,2025-10-09,260.0,9556.97,122,905,233943.55,GO,MAMBAI,ANIZIO FILHO,CENTRO-OESTE
4,0,11,24940522000131,2025-10-06,257.0,10247.04,92,902,200122.26,GO,MAMBAI,ANIZIO FILHO,CENTRO-OESTE


In [7]:
# Verificar informações do dataset
print("Informações do dataset:")
print("Shape do dataset:", df.shape)

print("\nDistribuição da variável target:")
print(df['ANTECIPA'].value_counts())
print("=" * 34)

Informações do dataset:
Shape do dataset: (435042, 13)

Distribuição da variável target:
ANTECIPA
0    410148
1     24894
Name: count, dtype: int64


In [8]:
df["UF"] = Dataframe.factorize(df["UF"])[0]
df["CIDADE"] = Dataframe.factorize(df["CIDADE"])[0]
df["GERENTE"] = Dataframe.factorize(df["GERENTE"])[0]
df["Região"] = Dataframe.factorize(df["Região"])[0]
df["ANTECIPA"] = df['ANTECIPA'].astype(int)

In [ ]:
df = df.sort_values(['CNPJ', 'DATA'])

df = Dataframe.generate_ema_grid(
  df, 
  groupby='CNPJ', 
  value_columns=['SALDO UNICA', 'TPV_CRED', 'QTD_TRN UNICA'], 
  windows=[7, 15, 30, 60, 90]
)

In [10]:
df = df.sort_values(['CIDADE', 'DATA'])

df = Dataframe.generate_ema_grid(
  df, 
  groupby='CIDADE', 
  value_columns=['SALDO UNICA', 'TPV_CRED', 'QTD_TRN UNICA'], 
  windows=[7, 15, 30, 60, 90]
)

In [11]:

# Converter para string primeiro, depois para datetime
df['DATA'] = Dataframe.to_datetime(df['DATA'].astype(str))

# Extrair features temporais
df['ano'] = df['DATA'].dt.year
df['mes'] = df['DATA'].dt.month
df['dia'] = df['DATA'].dt.day
df['dia_semana'] = df['DATA'].dt.dayofweek  # 0=Segunda, 6=Domingo

maxdata = df['DATA'].max() - timedelta(days=1)

c:\Users\henri\Desktop\mai\core\dataframe\__init__.py:68: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(


In [12]:
df = df.sort_values(['dia_semana', 'DATA'])

df = Dataframe.generate_ema_grid(
  df, 
  groupby='dia_semana', 
  value_columns=['SALDO UNICA', 'TPV_CRED', 'QTD_TRN UNICA'], 
  windows=[7, 15, 30, 60, 90]
)

In [13]:

testdf = df[df['DATA'] == maxdata ]
df = df[df['DATA'] < maxdata ]

In [14]:
df = df.drop('DATA', axis=1)
testdf = testdf.drop('DATA', axis=1)

In [ ]:
df.to_csv("./datasource/train_base.csv", index=False)
testdf.to_csv("./datasource/test_base.csv", index=False)

In [16]:
del df
del testdf